In [1]:
# pip install xgboost

import pandas as pd
import xgboost as xg

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('../Data/merged_data.csv').iloc[:,1:]

In [3]:
df.head()

,date,daily_rainfall_total,highest_30_min_rainfall,highest_60_min_rainfall,highest_120_min_rainfall,mean_temperature,maximum_temperature,minimum_temperature,mean_wind_speed,max_wind_speed,...,minimum_temperature_14,mean_wind_speed_14,max_wind_speed_14,aedes,dengue,fever,headache,nosebleed,vomit,recent_cases
0,2017-01-01,3.0,2.6,2.6,2.6,26.8,30.0,24.7,11.2,40.7,...,NaN,NaN,NaN,10.0,27.0,69.0,70.0,0.0,100.0,NaN
1,2017-01-02,47.2,32.6,42.2,45.2,26.1,30.5,24.1,6.1,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-03,0.6,0.6,0.6,0.6,26.3,30.6,24.5,7.6,27.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-04,2.6,2.4,2.4,2.4,26.6,30.2,23.5,9.0,33.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-05,1.2,0.8,1.0,1.2,27.7,31.4,24.4,8.6,33.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130926 entries, 0 to 130925
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   date                         130926 non-null  object 
 1   daily_rainfall_total         130926 non-null  float64
 2   highest_30_min_rainfall      130926 non-null  float64
 3   highest_60_min_rainfall      130926 non-null  float64
 4   highest_120_min_rainfall     130926 non-null  float64
 5   mean_temperature             130926 non-null  float64
 6   maximum_temperature          130926 non-null  float64
 7   minimum_temperature          130926 non-null  float64
 8   mean_wind_speed              130926 non-null  float64
 9   max_wind_speed               130926 non-null  float64
 10  year                         130926 non-null  int64  
 11  region                       130926 non-null  object 
 12  daily_rainfall_total_14      130107 non-null  float64
 13 

Assumption: Recent cases = 0 for blanks as our source data only collects info on clusters and does not inform us when an area is not a cluster. Data collection stopped after 8 Nov 2020.

In [153]:
df['recent_cases'].fillna(0, inplace = True)

We will be looking at weekly data instead of daily data. Thus, remove all na (since the google trends data is weekly)

In [154]:
df2 = df.dropna()

In [155]:
df2['month'] = df2['date'].str[5:7]
df2['month'] = df2['month'].astype(int)

C:\Users\jinmi\AppData\Local\Temp/ipykernel_29972/736178102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month'] = df2['date'].str[5:7]
C:\Users\jinmi\AppData\Local\Temp/ipykernel_29972/736178102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month'] = df2['month'].astype(int)


In [156]:
df2.columns

Index(['date', 'daily_rainfall_total', 'highest_30_min_rainfall',
       'highest_60_min_rainfall', 'highest_120_min_rainfall',
       'mean_temperature', 'maximum_temperature', 'minimum_temperature',
       'mean_wind_speed', 'max_wind_speed', 'year', 'region',
       'daily_rainfall_total_14', 'highest_30_min_rainfall_14',
       'highest_60_min_rainfall_14', 'highest_120_min_rainfall_14',
       'mean_temperature_14', 'maximum_temperature_14',
       'minimum_temperature_14', 'mean_wind_speed_14', 'max_wind_speed_14',
       'aedes', 'dengue', 'fever', 'headache', 'nosebleed', 'vomit',
       'recent_cases', 'month'],
      dtype='object')

In [157]:
df3 = df2[df2['date']<='2020-11-08']

In [158]:
df3.drop('date',axis=1,inplace=True)

In [166]:
df4 = pd.get_dummies(df3, columns=['region'], drop_first=True)
df4.head()

,daily_rainfall_total,highest_30_min_rainfall,highest_60_min_rainfall,highest_120_min_rainfall,mean_temperature,maximum_temperature,minimum_temperature,mean_wind_speed,max_wind_speed,year,...,region_Tengah,region_Toa Payoh,region_Tuas,region_Tuas South,region_Tuas West,region_Ulu Pandan,region_Upper Peirce Reservoir,region_Upper Thomson,region_Whampoa,region_Yishun
14,4.0,3.8,3.8,4.0,27.7,32.0,25.6,8.3,39.6,2017,...,0,0,0,0,0,0,0,0,0,0
21,1.6,0.8,1.4,1.4,26.2,30.1,23.9,12.6,43.9,2017,...,0,0,0,0,0,0,0,0,0,0
28,0.0,0.0,0.0,0.0,26.3,28.3,24.5,10.4,30.6,2017,...,0,0,0,0,0,0,0,0,0,0
35,0.0,0.0,0.0,0.0,27.3,32.6,24.9,9.7,27.4,2017,...,0,0,0,0,0,0,0,0,0,0
42,2.4,1.4,1.4,1.4,26.1,29.2,24.3,16.2,59.4,2017,...,0,0,0,0,0,0,0,0,0,0


In [160]:
df4.shape

(11494, 87)

In [161]:
df4.corr()['recent_cases'].sort_values(ascending=False).head(25)

recent_cases                      1.000000
region_Chai Chee                  0.206839
year                              0.169790
dengue                            0.159564
region_Ang Mo Kio                 0.138680
region_Tai Seng                   0.112204
region_Serangoon                  0.105495
aedes                             0.079513
region_Bukit Panjang              0.076713
nosebleed                         0.074726
region_Toa Payoh                  0.060581
region_Jurong (West)              0.051381
mean_wind_speed                   0.035175
mean_wind_speed_14                0.034026
region_Simei                      0.032227
region_Buangkok                   0.027099
mean_temperature                  0.025182
region_Marine Parade              0.024675
mean_temperature_14               0.024300
region_Choa Chu Kang (Central)    0.022071
region_Yishun                     0.019104
maximum_temperature               0.018805
minimum_temperature               0.018079
region_Wham

### Random Forest Model

In [168]:
X = df4.drop(['recent_cases'], axis=1)
y = df4['recent_cases']

In [169]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [170]:
rf = RandomForestRegressor()

In [176]:
rf_params = {
    'n_estimators': range(100,500,100),
    'max_depth': [5,7,10,20],
    'max_features': [20,30,35],
    'ccp_alpha': [0.001,0.01,0.05,0.1]
}
gs = GridSearchCV(rf, param_grid=rf_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

0.38125292463151933


{'ccp_alpha': 0.001, 'max_depth': 20, 'max_features': 30, 'n_estimators': 100}

In [177]:
r2_train = gs.score(X_train, y_train)
r2_test = gs.score(X_test, y_test)
rmse_train = mean_squared_error(y_train, gs.predict(X_train), squared = False)
rmse_test = mean_squared_error(y_test, gs.predict(X_test), squared = False)

print(f'R2 score on train set: {round(r2_train,5)}')
print(f'R2 score on test set: {round(r2_test,5)}')
print(f'RMSE score on train set: {round(rmse_train,5)}')
print(f'RMSE score on test set: {round(rmse_test,5)}')

R2 score on train set: 0.93482
R2 score on test set: 0.49241
RMSE score on train set: 7.72058
RMSE score on test set: 16.30511


Try tuning hyperparements to see if it helps with model improvement

In [195]:
rf_params = {
    'n_estimators': range(100,500,100),
    'max_depth': [10,20,50,100],
    'max_features': [10,20,30,35],
    'ccp_alpha': [0.001,0.005,0.01,0.05,0.1]
}
gs2 = GridSearchCV(rf, param_grid=rf_params, cv=3)
gs2.fit(X_train, y_train)
print(gs2.best_score_)
gs2.best_params_

0.38490384122900867


{'ccp_alpha': 0.01, 'max_depth': 20, 'max_features': 30, 'n_estimators': 100}

In [196]:
r2_train_2 = gs2.score(X_train, y_train)
r2_test_2 = gs2.score(X_test, y_test)
rmse_train_2 = mean_squared_error(y_train, gs2.predict(X_train), squared = False)
rmse_test_2 = mean_squared_error(y_test, gs2.predict(X_test), squared = False)

print(f'R2 score on train set: {round(r2_train_2,5)}')
print(f'R2 score on test set: {round(r2_test_2,5)}')
print(f'RMSE score on train set: {round(rmse_train_2,5)}')
print(f'RMSE score on test set: {round(rmse_test_2,5)}')

R2 score on train set: 0.9311
R2 score on test set: 0.5173
RMSE score on train set: 7.93769
RMSE score on test set: 15.9003


Model improved by a bit, but let's try XGBoost to see if we can get a better model

### XGBoost Model

In [197]:
xgb = xg.XGBRegressor()

In [198]:
xgb_params = {
    'subsample' : [0.3, 0.5, 0.7, 0.9, 1],
    'learning_rate' : [0.001, 0.01, 0.1, 0.3],
    'n_estimators' : range(100,500,100),
    'reg_lambda' : [50, 100, 1000, 10000],
    'reg_alpha' : [0, 1, 10, 100, 1000]
}
gs3 = GridSearchCV(xgb, param_grid=xgb_params, cv=3)
gs3.fit(X_train, y_train)
print(gs3.best_score_)
gs3.best_params_

0.3770442492197836


{'learning_rate': 0.3,
 'n_estimators': 400,
 'reg_alpha': 1,
 'reg_lambda': 1000,
 'subsample': 1}

In [199]:
r2_train_3 = gs3.score(X_train, y_train)
r2_test_3 = gs3.score(X_test, y_test)
rmse_train_3 = mean_squared_error(y_train, gs3.predict(X_train), squared = False)
rmse_test_3 = mean_squared_error(y_test, gs3.predict(X_test), squared = False)

print(f'R2 score on train set: {round(r2_train_3,5)}')
print(f'R2 score on test set: {round(r2_test_3,5)}')
print(f'RMSE score on train set: {round(rmse_train_3,5)}')
print(f'RMSE score on test set: {round(rmse_test_3,5)}')

R2 score on train set: 0.75825
R2 score on test set: 0.55865
RMSE score on train set: 14.86842
RMSE score on test set: 15.20412


Not fantastic, but lesser overfitting. Comparing all the models, the XGBoost is the best performing model.

In [204]:
test = pd.DataFrame({'feature':X.columns,'coefficient':gs3.best_estimator_.feature_importances_})

In [206]:
test.sort_values(by = 'coefficient', ascending = False).head(10)

,feature,coefficient
49,region_Tengah,0.103890
39,region_Paya Lebar,0.072038
23,region_Chai Chee,0.044318
17,region_Ang Mo Kio,0.037927
48,region_Tai Seng,0.037785
44,region_Serangoon,0.033234
42,region_Seletar,0.032758
18,region_Botanic Garden,0.029379
24,region_Changi,0.029242
27,region_Dhoby Ghaut,0.026228


Top 10 most important features to the model are the region features.